# 2. Manual and Automatic Brain State Scoring Concatenation 

This project aims to compare the accuracy of an automated machine learning model with human observation in the classification of rat brain states. 
The dataset used in this study comprises 50 rats. This markdown document details the process applied to one of these cases as an example. The data used for the automatic scoring was previously obtained from an automated ML scorer. 
The ML scorer's timeline aligns with the timeline from the human-scored data, facilitating a direct comparison between the automated model's outputs and human observations.

The steps are as follows:

1. **Gather Final Times of Recordings**: The final brain states are imported from an Excel file, specifically 'FINAL_BRAINSTATES.xlsx'.

2. **Import Automatic Score Data**: The data, from a CSV file named '433_BL1.csv', is the output of the automated brain state classification.

3. **Import Manual Score Data**: The manually classified brain states are imported from a pickle file named '433_score.pkl'.

4. **Isolate Brain State for Automatic Scoring**: The 'brainstate' column, indicating the brain state from automatic classification, is extracted.

5. **Concatenate Brain States**: The 'brainstate' column from the automatic classification is concatenated to the final brain states DataFrame.

6. **Modify Specific Brain State Values**: Certain 'brainstate' values are adjusted. For instance, 'brainstate' values equal to 3 are replaced with 4.

7. **Filter Time**: The data is filtered to retain only the records between 10 am to 2 pm.

8. **Reset and Rename Columns**: The DataFrame's index is reset. The 'brainstate' column is renamed to 'automatic_score' and columns not needed for the analysis are dropped.

9. **Modify Manual Score Data**: In the manual score data, 'brainstate' is renamed to 'manual_score' and irrelevant columns are dropped.

10. **Concatenate Manual and Automatic Scores**: The 'manual_score' and 'automatic_score' are concatenated into a single DataFrame.

11. **Drop Last Row**: The final row of the DataFrame, which is not required, is dropped.

12. **Export to CSV**: The consolidated DataFrame is exported as a CSV file named 'manual_auto_433.csv' for future use.


In [1]:
import pandas as pd
import numpy as np

In [2]:
#gather final times of recordings to use them as templates
final_brainstates = pd.read_excel('/Users/valentinreateguirangel/Python/Katerina_final_brainstates(excel)/FINAL_BRAINSTATES.xlsx', sheet_name ='Sheet2')
#import automatic score data 
automatic_score = pd.read_csv('/Users/valentinreateguirangel/Python/Automatic_scoring/433_BL1.csv')
#import manual score data
manual_score = pd.read_pickle('/Users/valentinreateguirangel/Python/Manual_scoring/Alex_scored/433_score.pkl')

In [3]:
#Isolating brain state only to copy it to final_brainstate by deault idex
auto_brainstate = automatic_score[['brainstate']]

In [4]:
#concatenating auto_brainstate values to final_brainstate into an new df
final_auto_brain = pd.concat([final_brainstates, auto_brainstate], axis=1)
final_auto_brain.loc[auto_brainstate['brainstate'] == 3, 'brainstate'] = 4

In [5]:
#Slicing data to get only the time from 10 am to 14 pm
target_time = final_auto_brain.loc[(final_auto_brain['ZT'] == 3)| 
                                    (final_auto_brain['ZT'] == 4)|
                                    (final_auto_brain['ZT'] == 5)|
                                    (final_auto_brain['ZT'] == 6),['ZT','LT','brainstate']]
#change the name of brainstate to recognise the automatic score for posterior steps
target_time= target_time.reset_index()
target_time = target_time.rename(columns={'brainstate': 'automatic_score'})
target_time= target_time.drop(['LT','ZT','index'], axis=1)

In [6]:
#values in the manual score 
manual_score = manual_score.rename(columns={'brainstate': 'manual_score'})
manual_score= manual_score.drop(['start_epoch','end_epoch'], axis=1)

In [7]:
#concatenating manual score and automatic score
manual_and_automatic = pd.concat([target_time, manual_score], axis = 1)
manual_and_automatic= manual_and_automatic.drop(manual_and_automatic.index[-1])

In [8]:
file_path = '/Users/valentinreateguirangel/Python/Manual_and_auto_score/manual_auto_433.csv'
manual_and_automatic.to_csv(file_path, index=False)